# Suggest word that i am writing now

In [2]:

# Define a list of common words and phrases
common_words = ["the", "and", "of", "to", "in", "a", "for", "is", "that",
"with", "on", "at", "by", "this", "you", "not", "be", "are", "from",
"or", "an", "but", "we", "can", "have", "as", "your", "all", "if",
"out", "when", "up", "there", "use", "so", "no", "do", "what", "my",
"which", "their", "about", "me", "will", "into", "like", "has", "them",
"than", "just", "other", "then", "would", "some", "could", "its",
"these", "two", "may", "many", "make", "most", "know", "see", "much",
"need", "our", "before", "such", "because", "very", "any", "now",
"over", "time", "only", "new", "years", "even", "year", "day", "people",
"after", "first", "last", "way", "well", "where", "get", "go", "back", "good",
"how", "down", "also", "two", "want", "between", "life", "work", "should", "never",
"great", "ever", "find", "long", "give", "always", "right", "world", "same", "still",
"think", "last", "high", "might", "end", "every", "while", "under", "such", "part",
"small", "few", "place", "own", "head", "house", "hand", "eyes", "face", "room",
"world", "water", "air", "earth", "fire","click" , "enter" , "tap" ]

def suggest_completions(prefix):
    completions = []
    prefix = prefix.lower()
    for word in common_words:
        if word.startswith(prefix):
            completions.append(word)
    completions.sort(key=lambda x: common_words.count(x), reverse=True)
    return completions[:1] # return the top 10 most common suggestions


In [4]:
suggest_completions("th")

['the']

# Suggest next word from phrase

In [5]:
import collections

# Define a dictionary of common phrases that follow specific words
phrases = {
    "I am": ["going to", "looking for", "thinking about"],
    "How are": ["you", "things", "work"],
    "Can you": ["help me", "give me", "tell me"],
    "What is": ["the weather", "your name", "going on"],
    "Where is": ["the bathroom", "the nearest store", "my phone"],
    "I need": ["to go", "some help", "a break"],
    "click": ["on the", "the button", "the link"],
}

# Define a deque to keep track of the last few words typed
# Define a deque to keep track of the last few words typed
last_words = collections.deque(maxlen=3)

def suggest_upcoming_words(current_text):
    # Split the current text into individual words
    words = current_text.split()
    
    # Add the last few words to the deque
    for word in words[-3:]:
        last_words.append(word)
    
    # Check if the last few words match a common phrase
    for phrase, options in phrases.items():
        if list(last_words)[-len(phrase.split()):] == phrase.split():
            return options
    
    return []

In [6]:
# Call suggest_upcoming_words with an example text
text = "click "
suggestions = suggest_upcoming_words(text)

# Print the suggestions
print("Suggestions for '{}':".format(text))
for suggestion in suggestions:
    print("- {}".format(suggestion))

Suggestions for 'click ':
- on the
- the button
- the link


In [8]:
# suggestion models 
# word_vectprs
# bigram
# trigram

## Trying N-gram Model for auto-complete depending on our data

### import required libraries

In [1]:
import pandas as pd
from collections import defaultdict

### Read The Data

In [2]:
# Load data into a Pandas DataFrame
data = pd.read_csv('../text Analyzer/ner_testing.csv')

### Preprocessing

In [3]:
# Preprocess the text data
# Define a function to preprocess the text data
def preprocess_text(text):
    # Remove any punctuation
    text = ''.join(c for c in text if c.isalnum() or c.isspace())
    # Convert to lowercase
    text = text.lower()
    # Split into individual words
    words = text.split()
    # Remove any stop words (optional)
    # words = [w for w in words if w not in stop_words]
    return words

# drop Asserion, Action, Target, Value columns
data = data.drop(['assertion', 'action', 'target', 'value'], axis=1)
data.head()


,text
0,the login button should be disabled
1,the title of the page should be equals to 'Hel...
2,the email input should have a value of hello@m...
3,you should see 2 tags of class tagClass
4,sould have 4 inputs of type text in the form


In [4]:
data['words'] = data['text'].apply(preprocess_text)
data.head()


,text,words
0,the login button should be disabled,"[the, login, button, should, be, disabled]"
1,the title of the page should be equals to 'Hel...,"[the, title, of, the, page, should, be, equals..."
2,the email input should have a value of hello@m...,"[the, email, input, should, have, a, value, of..."
3,you should see 2 tags of class tagClass,"[you, should, see, 2, tags, of, class, tagclass]"
4,sould have 4 inputs of type text in the form,"[sould, have, 4, inputs, of, type, text, in, t..."


### Build the model

In [5]:
def build_ngram_model(data, n=2):
    # Initialize a defaultdict to store the n-grams
    ngrams = defaultdict(list)
    
    # Loop through each row in the data
    for i, row in data.iterrows():
        # Get the list of words in the row
        words = row['words']
        
        # Loop through each 2-gram in the row and add it to the defaultdict 
        # Build the 2-gram model by adding word and its next word to the dictionary
        for j in range(len(words) - n + 1):
            ngram = ' '.join(words[j:j+n-1])
            next_word = words[j+n-1]
            ngrams[ngram].append(next_word)
    
    return ngrams

# Build the 2-gram model
ngrams = build_ngram_model(data, n=2)


In [20]:
import csv

with open("ngram.csv", 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ngram', 'next_word'])
        for ngram, next_words in ngrams.items():
            writer.writerow([ngram,  sorted(set(next_words), key=next_words.count, reverse=True)])

### Suggest word 

In [22]:
# Define a function to generate autocomplete suggestions
def suggest_words(text, ngrams, n=2):
    # Preprocess the text
    words = preprocess_text(text)
    
    # Get the most common next words for the 2-gram
    ngram = ' '.join(words[-n+1:]) if len(words) >= n else ''
    # print(ngram)
    if ngram in ngrams:
        next_words = ngrams[ngram]
        suggestions = sorted(set(next_words), key=next_words.count, reverse=True)
    else:
        suggestions = []
    
    return suggestions

In [38]:
suggest_words("click on button", ngrams, n=2)

['to',
 'should',
 'have',
 'is',
 'that',
 'will',
 'sort',
 'submit',
 'for',
 'in',
 'login']

In [51]:
words = preprocess_text("click on button")
data = pd.read_csv('ngram.csv')
data.set_index('ngram', inplace=True)
print("words:\n" , data.loc["button"]['next_word'])

words:
 ['to', 'should', 'have', 'is', 'that', 'will', 'sort', 'submit', 'for', 'in', 'login']
